# So sánh độ tương đồng hình ảnh bằng các hàm băm

## Mục tiêu
- So sánh các hình ảnh dựa trên **hàm băm ảnh**
- Sử dụng các phương pháp: aHash, dHash, pHash, wHash
- Đánh giá mức độ tương đồng bằng **khoảng cách Hamming**

## Ý tưởng chính
Ảnh được chuyển thành chuỗi bit (hash).  
Hai ảnh càng giống nhau thì số bit khác nhau càng ít.

In [1]:
import cv2
import numpy as np
import pywt
from scipy.fftpack import dct
import matplotlib.pyplot as plt


In [2]:
def show_image(image, title=""):
    plt.figure(figsize=(4,4))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis("off")
    plt.show()


In [3]:
def average_hash(image_path, hash_size=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (hash_size, hash_size))
    
    mean = image.mean()
    hash_bits = (image > mean).astype(np.uint8)
    
    return hash_bits.flatten()


In [4]:
def difference_hash(image_path, hash_size=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (hash_size + 1, hash_size))
    
    diff = image[:, 1:] > image[:, :-1]
    return diff.astype(np.uint8).flatten()


In [5]:
def perceptual_hash(image_path, hash_size=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (32, 32))
    
    dct_img = dct(dct(image.T, norm='ortho').T, norm='ortho')
    dct_low = dct_img[:hash_size, :hash_size]
    
    mean = dct_low.mean()
    return (dct_low > mean).astype(np.uint8).flatten()


In [6]:
def wavelet_hash(image_path, hash_size=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (256, 256))
    
    LL, _ = pywt.dwt2(image, 'haar')
    LL_small = cv2.resize(LL, (hash_size, hash_size))
    
    mean = LL_small.mean()
    return (LL_small > mean).astype(np.uint8).flatten()


In [7]:
def hamming_distance(hash1, hash2):
    return np.sum(hash1 != hash2)


In [8]:
image1 = "image1.jpg"
image2 = "image2.jpg"
image3 = "image3.jpg"

hash_functions = {
    "aHash": average_hash,
    "dHash": difference_hash,
    "pHash": perceptual_hash,
    "wHash": wavelet_hash
}

for name, func in hash_functions.items():
    h1 = func(image1)
    h2 = func(image2)
    dist = hamming_distance(h1, h2)
    print(f"{name}: Hamming Distance = {dist}")

print("--------------------------------")

for name, func in hash_functions.items():
    h1 = func(image1)
    h3 = func(image3)
    dist = hamming_distance(h1, h3)
    print(f"{name}: Hamming Distance = {dist}")


aHash: Hamming Distance = 23
dHash: Hamming Distance = 28
pHash: Hamming Distance = 19
wHash: Hamming Distance = 22
--------------------------------
aHash: Hamming Distance = 0
dHash: Hamming Distance = 0
pHash: Hamming Distance = 0
wHash: Hamming Distance = 0


## Đánh giá kết quả

| Hamming Distance | Mức độ tương đồng |
|------------------|------------------|
| 0 – 5 | Gần như giống nhau |
| 6 – 10 | Tương đồng cao |
| 11 – 20 | Có liên quan |
| > 20 | Khác nhau |


## So sánh các hàm băm

| Hàm băm | Ý tưởng | Độ chính xác | Tốc độ |
|-------|--------|-------------|--------|
| aHash | Trung bình | Thấp | Rất nhanh |
| dHash | Chênh lệch pixel | Trung bình | Rất nhanh |
| pHash | DCT | Cao | Chậm |
| wHash | Wavelet | Cao | Trung bình |


## Kết luận

Trong bài này, các hàm băm ảnh được sử dụng để trích xuất đặc trưng hình ảnh dưới dạng chuỗi bit.  
Khoảng cách Hamming cho phép đánh giá mức độ tương đồng giữa các hình ảnh.

Trong đó:
- aHash, dHash phù hợp cho bài toán đơn giản
- pHash và wHash cho độ chính xác cao hơn
- wHash là lựa chọn cân bằng giữa độ bền và tốc độ
